### This notebook is for capstone project

In [1]:
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Import all libraries


In [2]:

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

text = requests.get(url).text
html_parsed = BeautifulSoup(text, 'html.parser')

neightborhood_info_table = html_parsed.find('table', class_ = 'wikitable')
neightborhood_rows = neightborhood_info_table.find_all('tr')
neightborhood_info = []
for row in neightborhood_rows:
    info = row.text.split('\n')[1:-1]
    neightborhood_info.append(info)
neightborhood_info[0][-1] = 'Neighborhood'
neighborhood_df = pd.DataFrame(neightborhood_info[1:], columns=neightborhood_info[0])

#Cleaning Dataframe

no_boroughs = neighborhood_df.index[neighborhood_df['Borough'] == 'Not assigned']
no_neighborhoods = neighborhood_df.index[neighborhood_df['Neighborhood'] == 'Not assigned']
no_neighborhoods_or_borough = no_boroughs & no_neighborhoods


neighborhood_df.drop(neighborhood_df.index[no_boroughs], inplace=True)
neighborhood_df.reset_index(drop=True, inplace=True)

no_neighborhoods = neighborhood_df.index[neighborhood_df['Neighborhood'] == 'Not assigned'] 
for idx in no_neighborhoods:
    neighborhood_df['Neighborhood'][idx] = neighborhood_df['Borough'][idx]

group = neighborhood_df.groupby('Postcode')
grouped_neighborhoods = group['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_boroughs = group['Borough'].apply(lambda x: set(x).pop())
grouped_df = pd.DataFrame(list(zip(grouped_boroughs.index, grouped_boroughs, grouped_neighborhoods)))
grouped_df.columns = ['Postcode', 'Borough', 'Neighborhood']

grouped_df.head(12)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### 2. Convert the data from webpage into a Dataframe, Cleaning the data frame

In [3]:
grouped_df.shape

(103, 3)

In [4]:
coords_df = pd.read_csv('http://cocl.us/Geospatial_data')

new_grouped_df = grouped_df.join(coords_df.set_index('Postal Code'), on='Postcode')
new_grouped_df.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### 3. add lat and long for the postal codes

In [5]:
# Seperate locations only with toronto in them

kal = new_grouped_df[new_grouped_df.Borough.str.contains("Toronto")]
kal.reset_index(drop=True, inplace=True)

X = kal[['Longitude', 'Latitude']]

from geopy.distance import great_circle

#function to get the closest postcode pair and the distance using latitude and longitude
def closest_postcode(postcode, postcodes):
    postcode = np.asarray(postcode).reshape(1,-1)
    postcodes = np.asarray(postcodes)
    distances = [great_circle(postcode, point).meters for point in postcodes]
    closest_postcode_index = np.argmin(distances)
    return [round(distances[closest_postcode_index]), closest_postcode_index]

coordinates = kal[['Latitude', 'Longitude']]

distance, closest_point_index = closest_postcode(coordinates[:1], coordinates[~coordinates.index.isin([0])])
close_points = X[:1].append(X[closest_point_index+1:closest_point_index+2])


from math import isnan

for lat, lng, idx in zip(kal['Latitude'], kal['Longitude'], kal.index):
    distance, closest_point_index = closest_postcode([lat,lng], coordinates[~coordinates.index.isin([idx])])
    kal.at[idx,'Distance'] = np.int64(distance//2) #use the half of the disstance to avoid overlapping
    
kal.head(38)

print(kal.dtypes)


Postcode         object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
Distance        float64
dtype: object


/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
map = folium.Map(location=[43.6532,-79.3832], zoom_start=12)

for location in kal.itertuples(): #iterate each row of the dataframe
    label = 'Postal Code: {};  Borough: {};  Neighborhoods: {}'.format(location[1], location[2], location[3])
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [location[-3], location[-2]],
        radius=1,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
    folium.Circle(
        radius=location[-1],
        popup=label,
        location=[location[-3], location[-2]],
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(map) 
    
map

In [7]:
CLIENT_ID = 'Y2SDCM5ONE2XYX5RQPDJSYNJCNM3BQVGHF0Z0OCRDCWXKNZC' # your Foursquare ID
CLIENT_SECRET = 'QA4IVQZIZ4TLH4BZTZKWE3Y1JKGVL12SHMDE2IBP43VM45ZS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


#function to format url
def format_url(lat, lng, radius=500, limit=100):
    return 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 
        lat, lng, VERSION, radius, limit)

def get_venues(url):
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    return json_normalize(venues)

# postcode = kal.iloc[2, :]
# url = format_url(postcode['Latitude'], postcode['Longitude'], postcode['Distance'])
# print(url)

# venues_df = get_venues(url)


#function to clean the venues dataframe
def clean_df(df):
    relevant_columns = ['venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.name']
    clean_df = df.loc[:,relevant_columns]

    #rename columns
    clean_df.columns = ['Category', 'Lat', 'Lng', 'Name']
    return clean_df 

# venues_df = get_venues(url)

def get_category_name(row):
    category_json = row['Category']
    try:
        return category_json[0]['name']
    except:        
        return None
    
# venues_df = clean_df(venues_df)


#function that extract the category name
def get_category_name(row):
    category_json = row['Category']
    try:
        return category_json[0]['name']
    except:        
        return None

# venues_df['Category'] = venues_df.apply(get_category_name, axis=1)

In [8]:
def get_all_venues(postcodes, lat, lng, radius):
    venues_list=[]
    for postcode, lat, lng, radius in zip(postcodes, lat, lng, radius):
        url= format_url(lat, lng, radius)
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            postcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name'])
            for v in results])
    all_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    all_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category'
                  ]
    
    return all_venues

all_venues = get_all_venues(kal['Postcode'], kal['Latitude'], kal['Longitude'], kal['Distance'])

all_venues.head()

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,M4E,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
2,M4E,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,M4E,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,M4E,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery


In [9]:
print('{} venues were returned.'.format(all_venues.shape[0]))
print('there were {} unique venue categories'.format(len(all_venues['Venue Category'].unique())))

1596 venues were returned.
there were 253 unique venue categories


In [10]:
venues_in_each = all_venues.groupby('Postcode').count()
venues_in_each = pd.DataFrame(list(zip(venues_in_each.index, venues_in_each['Venue'])), columns=['Postcode', 'num of Venues']).set_index('Postcode').join(kal[['Postcode','Distance']].set_index('Postcode'), on='Postcode').reset_index()
venues_in_each['Distance'] = np.int64(venues_in_each['Distance'])

venues_in_each[['num of Venues']].describe()

,num of Venues
count,37.000000
mean,43.135135
std,33.118610
min,2.000000
25%,13.000000
50%,35.000000
75%,70.000000
max,100.000000


In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = all_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Tram Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [12]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(37, 254)


,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Tram Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.0000,0.000000,0.00,0.012987,0.025974,0.012987,0.038961,0.038961,0.000000,0.000000,0.012987,0.00,0.000000,0.00000,0.000000,0.012987,0.000000,0.000,0.038961,0.000000,0.000000,0.000000,0.025974,0.000000,0.000000,0.00000

In [13]:
num_top_venues = 5

for hood in toronto_grouped['Postcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
         venue  freq
0  Coffee Shop  0.06
1          Pub  0.06
2  Pizza Place  0.05
3          Bar  0.04
4        Beach  0.04


----M4K----
                  venue  freq
0      Greek Restaurant  0.13
1           Coffee Shop  0.07
2                   Pub  0.05
3                  Café  0.04
4  Fast Food Restaurant  0.04


----M4L----
                venue  freq
0         Pizza Place  0.06
1  Italian Restaurant  0.06
2       Burrito Place  0.06
3       Movie Theater  0.06
4          Steakhouse  0.06


----M4M----
            venue  freq
0          Bakery  0.06
1            Café  0.05
2     Coffee Shop  0.05
3             Bar  0.05
4  Sandwich Place  0.05


----M4N----
        venue  freq
0  Restaurant  0.11
1       Trail  0.11
2   Bookstore  0.11
3        Café  0.11
4    Pharmacy  0.11


----M4P----
               venue  freq
0     Breakfast Spot  0.14
1  Food & Drink Shop  0.14
2               Park  0.14
3     Clothing Store  0.14
4                Gym  0.14


----M4R----
    

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Bar,Park,Sandwich Place,Bakery
1,M4K,Greek Restaurant,Coffee Shop,Pub,Pizza Place,Café,Fast Food Restaurant,Grocery Store,Ice Cream Shop,Italian Restaurant,Ramen Restaurant
2,M4L,Pizza Place,Steakhouse,Movie Theater,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Fish & Chips Shop,Pub,Sandwich Place,Liquor Store
3,M4M,Bakery,Sandwich Place,Bar,Coffee Shop,Café,Gym,Brewery,Gastropub,Diner,American Restaurant
4,M4N,Bookstore,Gym / Fitness Center,Pharmacy,Café,Coffee Shop,Park,Restaurant,Bus Line,Trail,Donut Shop


In [16]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38] 

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [17]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = kal

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

toronto_merged.head(38) # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Distance,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,995.0,0.0,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Bar,Park,Sandwich Place,Bakery
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,896.0,0.0,Greek Restaurant,Coffee Shop,Pub,Pizza Place,Café,Fast Food Restaurant,Grocery Store,Ice Cream Shop,Italian Restaurant,Ramen Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,423.0,0.0,Pizza Place,Steakhouse,Movie Theater,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Fish & Chips Shop,Pub,Sandwich Place,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,799.0,0.0,Bakery,Sandwich Place,Bar,Coffee Shop,Café,Gym,Brewery,Gastropub,Diner,American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,851.0,0.0,Bookstore,Gym / Fitness Center,Pharmacy,Café,Coffee Shop,Park,Restaurant,Bus Line,Trail,Donut Shop
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,472.0,0.0,Breakfast Spot,Grocery Store,Food & Drink Shop,Park,Hotel,Clothing Store,Gym,Event Space,Ethiopian Restaurant,Discount Store
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,497.0,0.0,Coffee Shop,Yoga Studio,Bagel Shop,Park,Clothing Store,Chinese Restaurant,Dessert Shop,Salon / Barbershop,Diner,Mexican Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,472.0,0.0,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Thai Restaurant,Italian Restaurant,Coffee Shop,Deli / Bodega,Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,601.0,3.0,Park,Playground,Tennis Court,Gym,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,701.0,0.0,Coffee Shop,Italian Restaurant,Pizza Place,Pub,Sushi Restaurant,Gym,Skating Rink,Sandwich Place,Café,Bagel Shop


In [18]:
import math

# create map
map_clusters = folium.Map(location=[43.6532,-79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if( math.isnan(cluster) ):
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='#000',
            fill=True,
            fill_color='#1da0db',
            fill_opacity=0.7).add_to(map_clusters)
    else:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
       

map_clusters

In [19]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Distance,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,995.0,0.0,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Bar,Park,Sandwich Place,Bakery
1,East Toronto,896.0,0.0,Greek Restaurant,Coffee Shop,Pub,Pizza Place,Café,Fast Food Restaurant,Grocery Store,Ice Cream Shop,Italian Restaurant,Ramen Restaurant
2,East Toronto,423.0,0.0,Pizza Place,Steakhouse,Movie Theater,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Fish & Chips Shop,Pub,Sandwich Place,Liquor Store
3,East Toronto,799.0,0.0,Bakery,Sandwich Place,Bar,Coffee Shop,Café,Gym,Brewery,Gastropub,Diner,American Restaurant
4,Central Toronto,851.0,0.0,Bookstore,Gym / Fitness Center,Pharmacy,Café,Coffee Shop,Park,Restaurant,Bus Line,Trail,Donut Shop
5,Central Toronto,472.0,0.0,Breakfast Spot,Grocery Store,Food & Drink Shop,Park,Hotel,Clothing Store,Gym,Event Space,Ethiopian Restaurant,Discount Store
6,Central Toronto,497.0,0.0,Coffee Shop,Yoga Studio,Bagel Shop,Park,Clothing Store,Chinese Restaurant,Dessert Shop,Salon / Barbershop,Diner,Mexican Restaurant
7,Central Toronto,472.0,0.0,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Thai Restaurant,Italian Restaurant,Coffee Shop,Deli / Bodega,Restaurant
9,Central Toronto,701.0,0.0,Coffee Shop,Italian Restaurant,Pizza Place,Pub,Sushi Restaurant,Gym,Skating Rink,Sandwich Place,Café,Bagel Shop
11,Downtown Toronto,634.0,0.0,Coffee Shop,Restaurant,Park,Pizza Place,Japanese Restaurant,Diner,Gastropub,Café,Italian Restaurant,Bakery


In [20]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Distance,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,497.0,1.0,Ice Cream Shop,Garden,Yoga Studio,Discount Store,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Distance,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,740.0,2.0,Trail,Sushi Restaurant,Park,Jewelry Store,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Distance,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,601.0,3.0,Park,Playground,Tennis Court,Gym,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
10,Downtown Toronto,601.0,3.0,Park,Trail,Playground,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Dumpling Restaurant


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Distance,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,75.0,4.0,Coffee Shop,Restaurant,Deli / Bodega,Gym,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
